In [ ]:
# Install required libraries
!pip install alpaca-trade-api ta

# Import necessary libraries
from alpaca_trade_api.rest import REST, TimeFrame
import pandas as pd
from ta.trend import MACD
from ta.volatility import BollingerBands
from ta.momentum import KAMAIndicator
import pdb

# Alpaca API credentials
API_KEY = "PKJ76CJYBU6I8GP1S4WT"
API_SECRET = "ONRf25FhaLlkk7cwVdb9zp26xJGqIo3wfHjLsN0C"
BASE_URL = "https://paper-api.alpaca.markets"

# Initialize Alpaca REST client
alpaca = REST(API_KEY, API_SECRET, BASE_URL)
# Trading parametersO
SYMBOLS = ["AAPL","MSFT","NVDA","AMZN","GOOG","META","TSLA","AVGO","WMT","JPM","V","XOM","ORCL","MA","COST","HD","PG","NFLX","JNJ","BAC","ABBV","CRM","TMUS","KO","WFC","CSCO","BX","ACN","ADBE","AMD","AXP","DIS","ASML","KR","F","PCG"]  # Stock symbols to trade
QUANTITY = 200  # Number of shares per trade
CASH_THRESHOLD = 1000  # Minimum cash to execute a trade

def fetch_data(symbol, timeframe, limit):
    """
    Fetch historical data from Alpaca.
    """
    bars = alpaca.get_bars(symbol, timeframe, limit=limit).df
    #bars = bars[bars['symbol'] == symbol]  # Filter for the specific symbol
    return bars

def calculate_indicators(data):
    """Calculate MACD, Squeeze Momentum, and Bollinger Bands indicators."""
    # MACD
    macd = MACD(data['close'], window_slow=26, window_fast=12, window_sign=9)
    data['macd'] = macd.macd()
    data['macd_signal'] = macd.macd_signal()

    # Squeeze Momentum (using KAMA as a proxy)
    kama = KAMAIndicator(data['close'], window=20, pow1=2, pow2=30)
    data['squeeze_momentum'] = kama.kama()

    # Bollinger Bands
    try:
        bb = BollingerBands(data['close'], window=20, window_dev=2)
        data['bb_upper'] = bb.bollinger_hband()
        data['bb_lower'] = bb.bollinger_lband()
        data['bb_middle'] = bb.bollinger_mavg()

        # Ensure there are no missing values in the Bollinger Bands
        data[['bb_upper', 'bb_lower', 'bb_middle']] = data[['bb_upper', 'bb_lower', 'bb_middle']].fillna(0)

        # Calculate Bollinger Band Width
        data['bb_width'] = (data['bb_upper'] - data['bb_lower']) / data['bb_middle']
    except Exception as e:
        print(f"Error in Bollinger Bands calculation: {e}")
        data['bb_upper'] = data['bb_lower'] = data['bb_middle'] = data['bb_width'] = 0

    # Average True Range (ATR) for consolidation
    data['atr'] = data['high'] - data['low']

    return data

def check_signals(data):
    """
    Determine buy or sell signals based on indicators.
    """
    latest = data.iloc[-1]
    previous = data.iloc[-2]

    # Check for consolidation (Bollinger Band width or ATR too narrow)
    is_consolidating = latest['bb_width'] < 0.01 * latest['close'] or latest['atr'] < 0.005 * latest['close']

    buy_signal = (
        not is_consolidating  # Ensure stock is not in consolidation
        and latest['macd'] > latest['macd_signal']  # MACD above signal line
        and latest['squeeze_momentum'] > 0  # Squeeze Momentum positive
        and previous['macd'] <= previous['macd_signal']  # Previous MACD below signal line
    )
    sell_signal = (
        not is_consolidating  # Ensure stock is not in consolidation
        and latest['macd'] < latest['macd_signal']  # MACD below signal line
        and latest['squeeze_momentum'] < 0  # Squeeze Momentum negative
        and previous['macd'] >= previous['macd_signal']  # Previous MACD above signal line
    )
    return buy_signal, sell_signal

def place_order(symbol, quantity, side):
    """
    Place a buy or sell order.
    """
    try:
        order = alpaca.submit_order(
            symbol=symbol,
            qty=quantity,
            side=side,
            type="market",
            time_in_force="gtc"
        )
        print(f"Order placed: {side} {quantity} shares of {symbol}")
    except Exception as e:
        print(f"Error placing order: {e}")

def run_strategy(symbol):
    """Main trading strategy loop."""
    try:
        # Fetch data
        data = fetch_data(symbol=symbol, timeframe=TimeFrame.Minute, limit=50)

        if len(data) < 2:
            print(f"Not enough data for {symbol}. Skipping...")
            return

        # Calculate indicators
        data = calculate_indicators(data)

        # Debugging: Check if `bb_width` exists in the DataFrame
        if 'bb_width' not in data.columns:
            print(f"Error: 'bb_width' column is missing for {symbol}. Skipping...")
            return

        # Check account for cash
        account = alpaca.get_account()
        cash = float(account.cash)

        # Check signals
        buy_signal, sell_signal = check_signals(data)
        if buy_signal and cash > CASH_THRESHOLD:
            place_order(symbol=symbol, quantity=QUANTITY, side="buy")
        elif sell_signal:
            place_order(symbol=symbol, quantity=QUANTITY, side="sell")
    except Exception as e:
        print(f"Error in strategy for {symbol}: {e}")
        pdb.post_mortem

# Run the strategy
#while True :
for SYMBOL in SYMBOLS:
  if __name__ == "__main__":
      import time
      run_strategy(SYMBOL)
#time.sleep(60)  # Wait for a minute before checking again
